Призначення скрипту: знаходження фрагменту по шаблону (pattern) у pdf-документі і заміни його на алтернативний (alter). Розрахований на односторінковий документ. Задіяні інструменти `fitz` (для роботи з pdf), `OpenCV` (для пошуку фрагмента інвойсу, що схожий на шаблон та заміну його на альтернативний), `Numpy`.

Для встановлення сторонніх бібліотек небхідно ввести:


```
pip install pymupdf, opencv-python, numpy
```
ВАЖЛИВО!

Шуканий та замінний шаблони мають бути зроблені завчасно і зберігатися у теці `Templates`. При цьому вони повинні бути зроблені з дотримнням масштабності. [Тут](https://colab.research.google.com/drive/1oEWHce6M3FBNgiSBHZs6XJf6Ty723QE1#scrollTo=E2i_7QGG8bBU&line=6&uniqifier=1) знаходиться скрипт для отримання зображення з документу щоб робити з нього шаблони.

Перед запуском обов'язково потрібно зазначити шляхи до тек із текою, з якої запускається скрипт (CURRENT_DIR), документами, які змінюємо (DOCUMENTS), шаблонами (TEMPLATES) та результатами (RESULT_DIR) у ячейці `Credentials`.  


У даному випадку схема шляхів до тек виглядає наступним чином:
```
CURRENT_DIR
|_DOCUMENTS
  |_RESULT_DIR
|_TEMPLATES
```

In [ ]:
!pip install --upgrade pymupdf
import cv2 as cv
import numpy as np
import fitz
from matplotlib import pyplot as plt
from google.colab.patches import cv2_imshow
from google.colab import drive
import os
drive.mount('/gdrive')

# Credentials

In [ ]:
# credentials
CURRENT_DIR = "/gdrive/MyDrive/"
DOCUMENTS = "/gdrive/MyDrive/"
TEMPLATES = CURRENT_DIR + "Templates/"
RESULT_DIR = DOCUMENTS + "RESULTS/"

## WORKING CELL

In [ ]:
%%time
# bound source pattern with alternative
patterns_img = {('pattern1.png', 'alter_1img.png'), ('pattern2.png', 'alter_2img.png'), ('pattern3.png', 'alter_3img.png')}
patterns_text = {('pattern1.png', 'alter_1txt.png'), ('pattern2.png', 'alter_2txt.png'), ('pattern3.png', 'alter_3txt.png')}
print('Process: ', end='')


def get_file() -> str:
    '''Iterate files in the dir and return filename without path
    Return: filenames -> generator'''
    for file in os.listdir(DOCUMENTS):
        if file.endswith(".pdf"):
            yield file

count = 0
warning = []
for pdf_file in get_file():
    doc = fitz.open(DOCUMENTS + pdf_file)
    # get picture from pdf
    page = doc.load_page(0)
    pil_image = page.get_pixmap(dpi=200)
    byte_img = pil_image.pil_tobytes('png')
    np_image = np.frombuffer(byte_img, np.uint8)
    invoice_img = cv.imdecode(np_image, 0)
    # determines inner format of document by size of file 
    if  os.path.getsize(DOCUMENTS + pdf_file) < 300000:
        pattern_files = patterns_text
    else:
        pattern_files = patterns_img

    # replace templates
    for filenames in pattern_files:
        tik = "|"
        # get template
        template = cv.imread(TEMPLATES + filenames[0], 0)
        w, h = template.shape[::-1]
        res = cv.matchTemplate(invoice_img, template, cv.TM_CCOEFF_NORMED)
        # check if the template matches (0.8 is threshold)
        if np.amax(res) < 0.8:
            warning.append(f'Template \"{filenames[0]}\" missmatched with source: \"{doc.name}\"\n')
            pdf_file = "$" + pdf_file
            tik = "0"
            break
        min_val, max_val, min_loc, max_loc = cv.minMaxLoc(res)
        top_left = max_loc
        
        # mark found fragment by rectangle
        bottom_right = (top_left[0] + w, top_left[1] + h)
        cv.rectangle(invoice_img, top_left, bottom_right, 255, -1)

        # get alter
        alter = cv.imread(TEMPLATES + filenames[1], 0)
        # define POI
        h_top = top_left[1]
        h_bottom = h_top + alter.shape[::-1][1]
        w_left = top_left[0]
        w_right = w_left + alter.shape[::-1][0]
        # insert alterimage in в POI
        invoice_img[h_top : h_bottom, w_left : w_right] = alter

    #save results
    if not os.path.isdir(RESULT_DIR):
        os.mkdir(RESULT_DIR)
    res, im_png = cv.imencode('.png', invoice_img)
    bytes_img = im_png.tobytes()
    doc.delete_page(0)
    doc.new_page()
    page = doc.load_page(0)
    page.insert_image(page.rect, stream=bytes_img)
    doc.save(RESULT_DIR + pdf_file)
    count += 1
    print(tik, end='')

# print('\nTime: ', round(time.time() - start_prog), 'sec')
print('\nTotal files: ', count)
print('Problem moments: ', len(warning))
for i in warning:
    print(i)
     

# Sandbox

Витягуємо зображення документу для створення шуканого (pattern) або альтернативного (alter) шаблону

In [ ]:
doc = fitz.open("/gdrive/MyDrive/document1.pdf")
page = doc.load_page(0)
pil_image = page.get_pixmap(dpi=200, colorspace='RGB')
byte_img = pil_image.pil_tobytes('png')
np_image = np.frombuffer(byte_img, np.uint8)
invoice_img = cv.imdecode(np_image, 0)
print(invoice_img.shape[::-1])
# cv2_imshow(invoice_img)
pil_image.pil_save('document1.png')

(1653, 2337)


Робимо заміну  шаблонів у одному інвойсі

In [ ]:
doc = fitz.open("/gdrive/MyDrive/document1.pdf")

patterns_img = {('pattern1.png', 'alter_1img.png'), ('pattern2.png', 'alter_2img.png'), ('pattern3.png', 'alter_3img.png')}
patterns_text = {('pattern1.png', 'alter_1txt.png'), ('pattern2.png', 'alter_2txt.png'), ('pattern3.png', 'alter_3txt.png')}
page = doc.load_page(0)
pil_image = page.get_pixmap(dpi=200)
byte_img = pil_image.pil_tobytes('png')
# np_image = np.frombuffer(bytearray(byte_img), dtype=np.uint8)
np_image = np.frombuffer(byte_img, np.uint8)
invoice_img = cv.imdecode(np_image, 0)
print(invoice_img.shape[::-1])

if  os.path.getsize(INVOICES + pdf_file) < 300000:
    pattern_files = patterns_text
else:
    pattern_files = patterns_img

for filenames in pattern_files:
    template = cv.imread('/gdrive/MyDrive/GB/ + filenames[0], 0)
    w, h = template.shape[::-1]
    res = cv.matchTemplate(invoice_img, template, cv.TM_CCOEFF_NORMED)
    # check if the template matches
    if np.amax(res) < 0.8:
        print(f'Template \"{filenames[0]}\" missmatched with source: \"{doc.name}\"')
        continue
    min_val, max_val, min_loc, max_loc = cv.minMaxLoc(res)
    top_left = max_loc
    # позначаємо знайдений фрагмент
    bottom_right = (top_left[0] + w, top_left[1] + h)
    cv.rectangle(invoice_img, top_left, bottom_right, 255, -1)

    alter = cv.imread(TEMPLATES + filenames[1], 0)
    # define POI
    h_top = top_left[1]
    h_bottom = h_top + alter.shape[::-1][1]
    w_left = top_left[0]
    w_right = w_left + alter.shape[::-1][0]
    # Вставляємо альтернативне зображення в POI
    invoice_img[h_top : h_bottom, w_left : w_right] = alter


cv2_imshow(invoice_img)


In [ ]:
res, im_png = cv.imencode('.png', invoice_img)
bytes_img = im_png.tobytes()
doc.delete_page(0)
doc.new_page()
page = doc.load_page(0)
page.insert_image(page.rect, stream=bytes_img,  width=1449, height=2048)
doc.save('new_samp1.pdf')

Підстановка лише одного шаблону

In [ ]:
doc = fitz.open("/gdrive/MyDrive/document.pdf")

page = doc.load_page(0)
pil_image = page.get_pixmap(dpi=200, colorspace='csGRAY')
byte_img = pil_image.pil_tobytes('png')
# np_image = np.frombuffer(bytearray(byte_img), dtype=np.uint8)
np_image = np.frombuffer(byte_img, np.uint8)
invoice_img = cv.imdecode(np_image, 0)
print(invoice_img.shape[::-1])


template = cv.imread(TEMPLATES + 'pattern2.png',0)
alter = cv.imread(TEMPLATES + 'alter2.png',0)
w, h = template.shape[::-1]
res = cv.matchTemplate(invoice_img, template, cv.TM_CCOEFF_NORMED)
min_val, max_val, min_loc, max_loc = cv.minMaxLoc(res)
top_left = max_loc
# позначаємо знайдений фрагмент
# bottom_right = (top_left[0] + w, top_left[1] + h)
# cv.rectangle(invoice_img, top_left, bottom_right, 0, -1)

# define POI
h_top = top_left[1]
h_bottom = h_top + alter.shape[::-1][1]
w_left = top_left[0]
w_right = w_left + alter.shape[::-1][0]

# invoice_img[h_top : h_bottom, w_left : w_right+12] = alter
invoice_img[h_top : h_bottom, w_left : w_right] = alter


print(cv.minMaxLoc(invoice_img))
# print(roi)
cv2_imshow(invoice_img)
# fig, ax = plt.subplots()
# fig.set_size_inches(20,20)
# plt.imshow(invoice_img)

##Витягуємо зображення з pdf за допомогою fitz і передаємо його cv

Метод №1 через doc.extract_image(xref)

In [ ]:
doc = fitz.open("/gdrive/MyDrive/doc_for_alters.pdf")
for xref in range(1, doc.xref_length()-1):
  if doc.xref_is_image:
      img_xref = doc.extract_image(xref)
      break #break, we are interested only in first image  
image_bytes = img_xref['image']
image_np = np.frombuffer(image_bytes, np.uint8)
img = cv.imdecode(image_np, 0)
w, h = img.shape[::-1]
print(w, h)
cv.imwrite('invoice_for_alters.png',img)
# cv2_imshow(img_np)

Метод №2 через get_pixmap (кращий метод). Цей метод переторює всю сторінку у зображення розміром (1653, 2337) при 200 dpi

In [ ]:
import io
doc = fitz.open("/gdrive/MyDrive/for_alters_text.pdf")

page = doc.load_page(0)
pil_image = page.get_pixmap(dpi=200, colorspace='RGB')
byte_img = pil_image.pil_tobytes('png')
np_image = np.frombuffer(byte_img, np.uint8)
invoice_img = cv.imdecode(np_image, 0)
print(invoice_img.shape[::-1])
# cv2_imshow(invoice_img)
pil_image.pil_save('/gdrive/MyDrive/alters_text.png')

(1654, 2339)
